In [1]:
#Los 3 primeros imports son para llamar a la API de ChatGPT
from dotenv import load_dotenv
import os #provides ways to access the Operating System and allows us to read the environment variables
import openai

import requests #Este import es para llamar a la API del tiempo

import pandas as pd #Este import es para cargar el CSV de latitud y longitud de todos los pueblos de españa

from haversine import haversine, Unit #Estos import son para calcular las distancias de lat y long
from math import radians, sin, cos, sqrt, atan2

import streamlit as st #Es para poder crear la APP
from PIL import Image #cargar el logo

In [2]:
origin_places = pd.read_csv('./data/municipios_list.csv', encoding='latin-1', sep=';')
origin_places[:2]

,Pueblo,Provincia
0,A Baña,(A Coruña)
1,A Capela,(A Coruña)


In [3]:
select_origin = origin_places.values.tolist()
select_origin

[['A Baña', '(A Coruña)'],
 ['A Capela', '(A Coruña)'],
 ['A Coruña', '(A Coruña)'],
 ['A Laracha', '(A Coruña)'],
 ['A Pobra do Caramiñal', '(A Coruña)'],
 ['Abegondo', '(A Coruña)'],
 ['Ames', '(A Coruña)'],
 ['Aranga', '(A Coruña)'],
 ['Ares', '(A Coruña)'],
 ['Arteixo', '(A Coruña)'],
 ['Arzúa', '(A Coruña)'],
 ['As Pontes de García Rodríguez', '(A Coruña)'],
 ['As Somozas', '(A Coruña)'],
 ['Bergondo', '(A Coruña)'],
 ['Betanzos', '(A Coruña)'],
 ['Boimorto', '(A Coruña)'],
 ['Boiro', '(A Coruña)'],
 ['Boqueixón', '(A Coruña)'],
 ['Brión', '(A Coruña)'],
 ['Cabana de Bergantiños', '(A Coruña)'],
 ['Cabanas', '(A Coruña)'],
 ['Camariñas', '(A Coruña)'],
 ['Cambre', '(A Coruña)'],
 ['Carballo', '(A Coruña)'],
 ['Cariño', '(A Coruña)'],
 ['Carnota', '(A Coruña)'],
 ['Carral', '(A Coruña)'],
 ['Cedeira', '(A Coruña)'],
 ['Cee', '(A Coruña)'],
 ['Cerceda', '(A Coruña)'],
 ['Cerdido', '(A Coruña)'],
 ['Coirós', '(A Coruña)'],
 ['Corcubión', '(A Coruña)'],
 ['Coristanco', '(A Coruña)'],


In [4]:
user_selection = input("Select the origin: ")

Select the origin: A Baña


In [5]:
#Primer para streamlit

#image = Image.open('./data/camp_n_rolla.jpg') #logo
#st.image(image, width=250) #logo
#st.title("Camp :red[N] Rolla") #titulo
#st.header("Hey Roller friend!") #frase bienvenida
#st.subheader("_Are you ready for a last minute roadtrip in your campervan or RV?_") #subtitulo

In [6]:
origen = pd.read_csv('./data/municipios_punto.csv')
origen

,Pueblo,Provincia,Longitud,Latitud,Datos de Interés
0,Alegría-Dulantzi,Álava,-2.512437,42.839812,NaN
1,Amurrio,Álava,-3.000073,43.054278,NaN
2,Aramaio,Álava,-2.565400,43.051197,NaN
3,Artziniega,Álava,-3.127917,43.120844,NaN
4,Armiñón,Álava,-2.871835,42.723262,NaN
...,...,...,...,...,...
8072,Biel,Zaragoza,-0.936572,42.387568,NaN
8073,Marracos,Zaragoza,-0.776416,42.090917,NaN
8074,Villamayor de Gállego,Zaragoza,-0.770533,41.684198,NaN
8075,Ceuta,Ceuta,-5.306751,35.888102,NaN


In [7]:
user_select = origen[origen['Pueblo'] == user_selection]
user_select

,Pueblo,Provincia,Longitud,Latitud,Datos de Interés
2132,A Baña,A Coruña,-8.758003,42.961898,NaN


In [8]:
user_idx = user_select.index[user_select['Pueblo'] == user_selection][0]
user_idx

2132

In [9]:
lat_origen = user_select['Latitud'][user_idx]
long_origen = user_select['Longitud'][user_idx]
print(lat_origen)
long_origen

42.96189751


-8.758002564

In [30]:
user_distance = int(input("How many km do you want to travel?: "))

How many km do you want to travel?: 15


In [31]:
def get_coordinates_south(latitude, longitude, distance_km):

    distance_meters = distance_km * 1000 #convierto a metros

    #Me devuelve las coordenadas de destino usando haversine 
    destination_lat = latitude - (distance_meters / 1000) / 111.32
    destination_lon = longitude

    return destination_lat, destination_lon

In [32]:
def get_coordinates_north(latitude, longitude, distance_km):

    distance_meters = distance_km * 1000

    destination_lat = latitude + (distance_meters / 1000) / 111.32
    destination_lon = longitude

    return destination_lat, destination_lon

In [33]:
def get_coordinates_east(latitude, longitude, distance_km):
    
    distance_meters = distance_km * 1000

    destination_lat = latitude
    destination_lon = longitude + (distance_meters / 1000) / (111.32 * abs(latitude))

    return destination_lat, destination_lon

In [34]:
def get_coordinates_west(latitude, longitude, distance_km):

    distance_meters = distance_km * 1000

    destination_lat = latitude
    destination_lon = longitude - (distance_meters / 1000) / (111.32 * abs(latitude))

    return destination_lat, destination_lon

In [35]:
def get_coordinates_northeast(latitude, longitude, distance_km):
    distance_meters = distance_km * 1000

    destination_lat = latitude + (distance_meters / 1000) / 111.32
    destination_lon = longitude + (distance_meters / 1000) / (111.32 * abs(latitude))

    return destination_lat, destination_lon

In [36]:
def get_coordinates_northwest(latitude, longitude, distance_km):
    
    distance_meters = distance_km * 1000

    destination_lat = latitude + (distance_meters / 1000) / 111.32
    destination_lon = longitude - (distance_meters / 1000) / (111.32 * abs(latitude))

    return destination_lat, destination_lon

In [37]:
def get_coordinates_southeast(latitude, longitude, distance_km):

    distance_meters = distance_km * 1000

    destination_lat = latitude - (distance_meters / 1000) / 111.32
    destination_lon = longitude + (distance_meters / 1000) / (111.32 * abs(latitude))

    return destination_lat, destination_lon

In [38]:
def get_coordinates_southwest(latitude, longitude, distance_km):

    distance_meters = distance_km * 1000

    destination_lat = latitude - (distance_meters / 1000) / 111.32
    destination_lon = longitude - (distance_meters / 1000) / (111.32 * abs(latitude))

    return destination_lat, destination_lon

In [39]:
if __name__ == "__main__":
    original_latitude = lat_origen
    original_longitude = long_origen
    distance_km = user_distance
    
    # Calculate coordinates for various directions
    south_lat, south_lon = get_coordinates_south(original_latitude, original_longitude, distance_km)
    north_lat, north_lon= get_coordinates_north(original_latitude, original_longitude, distance_km)
    east_lat, east_lon = get_coordinates_east(original_latitude, original_longitude, distance_km)
    west_lat, west_lon = get_coordinates_west(original_latitude, original_longitude, distance_km)
    northeast_lat, northeast_lon = get_coordinates_northeast(original_latitude, original_longitude, distance_km)
    northwest_lat, northwest_lon = get_coordinates_northwest(original_latitude, original_longitude, distance_km)
    southeast_lat, southeast_lon = get_coordinates_southeast(original_latitude, original_longitude, distance_km)
    southwest_lat, southwest_lon = get_coordinates_southwest(original_latitude, original_longitude, distance_km)

    # Create a DataFrame to store the results
    df_destino = pd.DataFrame({
        'Direction': ['South', 'North', 'East', 'West', 'Northeast', 'Northwest', 'Southeast', 'Southwest'],
        'Latitude': [south_lat, north_lat, east_lat, west_lat, northeast_lat, northwest_lat, southeast_lat, southwest_lat],
        'Longitude': [south_lon, north_lon, east_lon, west_lon, northeast_lon, northwest_lon, southeast_lon, southwest_lon]
    })

    print(df_destino)

   Direction   Latitude  Longitude
0      South  42.827151  -8.758003
1      North  43.096644  -8.758003
2       East  42.961898  -8.754866
3       West  42.961898  -8.761139
4  Northeast  43.096644  -8.754866
5  Northwest  43.096644  -8.761139
6  Southeast  42.827151  -8.754866
7  Southwest  42.827151  -8.761139


In [40]:
user_direction_idx_s = df_destino.index[df_destino['Direction'] == 'South'][0]
user_direction_idx_n = df_destino.index[df_destino['Direction'] == 'North'][0]
user_direction_idx_e = df_destino.index[df_destino['Direction'] == 'East'][0]
user_direction_idx_w = df_destino.index[df_destino['Direction'] == 'West'][0]
user_direction_idx_ne = df_destino.index[df_destino['Direction'] == 'Northeast'][0]
user_direction_idx_nw = df_destino.index[df_destino['Direction'] == 'Northwest'][0]
user_direction_idx_se = df_destino.index[df_destino['Direction'] == 'Southeast'][0]
user_direction_idx_sw = df_destino.index[df_destino['Direction'] == 'Southwest'][0]

In [41]:
lat_destino_s = df_destino['Latitude'][user_direction_idx_s]
long_destino_s = df_destino['Longitude'][user_direction_idx_s]
lat_destino_n = df_destino['Latitude'][user_direction_idx_n]
long_destino_n = df_destino['Longitude'][user_direction_idx_n]
lat_destino_e = df_destino['Latitude'][user_direction_idx_e]
long_destino_e = df_destino['Longitude'][user_direction_idx_e]
lat_destino_w = df_destino['Latitude'][user_direction_idx_w]
long_destino_w = df_destino['Longitude'][user_direction_idx_w]
lat_destino_ne = df_destino['Latitude'][user_direction_idx_ne]
long_destino_ne = df_destino['Longitude'][user_direction_idx_ne]
lat_destino_nw = df_destino['Latitude'][user_direction_idx_nw]
long_destino_nw = df_destino['Longitude'][user_direction_idx_nw]
lat_destino_se = df_destino['Latitude'][user_direction_idx_se]
long_destino_se = df_destino['Longitude'][user_direction_idx_se]
lat_destino_sw = df_destino['Latitude'][user_direction_idx_sw]
long_destino_sw = df_destino['Longitude'][user_direction_idx_sw]

In [42]:
ws = str(lat_destino_s) + ',' + str(long_destino_s)
wn = str(lat_destino_n) + ',' + str(long_destino_n)
we = str(lat_destino_e) + ',' + str(long_destino_e)
ww = str(lat_destino_w) + ',' + str(long_destino_w)
wne = str(lat_destino_ne) + ',' + str(long_destino_ne)
wnw = str(lat_destino_nw) + ',' + str(long_destino_nw)
wse = str(lat_destino_se) + ',' + str(long_destino_se)
wsw = str(lat_destino_sw) + ',' + str(long_destino_sw)

In [43]:
#Creo todas las variables para llamar a la API del tiempo menos "q" que representa el lugar 
#de dónde quiero ver la previsión del tiempo
load_dotenv()
url = 'http://api.weatherapi.com/v1'
forecast = '/forecast.json?'
pre = 'key='
key =os.getenv("my_weather_token")
wdays = '&days=1'
final = '&aqi=no&alerts=no'
q_var = '&q='

In [44]:
ws_weather = q_var + ws
wn_weather = q_var + wn
we_weather = q_var + we
ww_weather = q_var + ww
wne_weather = q_var + wne
wnw_weather = q_var + wnw
wse_weather = q_var + ws
wsw_weather = q_var + wsw

In [45]:
#Creo una llamada a la API para cada lugar

check_rain_north = requests.get(url + forecast + pre + key + wn_weather + wdays + final)
check_rain_nortwest = requests.get(url + forecast + pre + key + wnw_weather + wdays + final)
check_rain_west = requests.get(url + forecast + pre + key + ww_weather + wdays + final)
check_rain_southwest = requests.get(url + forecast + pre + key + wsw_weather + wdays + final)
check_rain_south = requests.get(url + forecast + pre + key + ws_weather + wdays + final)
check_rain_southeast = requests.get(url + forecast + pre + key + wse_weather + wdays + final)
check_rain_east = requests.get(url + forecast + pre + key + we_weather + wdays + final)
check_rain_northeast = requests.get(url + forecast + pre + key + wne_weather + wdays + final)

In [46]:
#Reviso si llueve en cada dirección

check_rain_north_result = check_rain_north.json()
check_rain_nortwest_result = check_rain_nortwest.json()
check_rain_west_result = check_rain_west.json()
check_rain_southwest_result = check_rain_southwest.json() 
check_rain_south_result = check_rain_south.json() 
check_rain_southeast_result = check_rain_southeast.json()
check_rain_east_result = check_rain_east.json()
check_rain_northeast_result = check_rain_northeast.json()

In [47]:
def not_raining_places():

    placestogo = []

    if check_rain_north_result['forecast']["forecastday"][0]['day']['daily_chance_of_rain'] == 0:
        placestogo.append('North')
    if check_rain_nortwest_result['forecast']["forecastday"][0]['day']['daily_chance_of_rain'] == 0:
        placestogo.append('Northwest')
    if check_rain_west_result['forecast']["forecastday"][0]['day']['daily_chance_of_rain'] == 0: 
        placestogo.append('West')  
    if check_rain_southwest_result['forecast']["forecastday"][0]['day']['daily_chance_of_rain'] == 0: 
        placestogo.append('Southwest')
    if check_rain_south_result['forecast']["forecastday"][0]['day']['daily_chance_of_rain'] == 0:
        placestogo.append('South')
    if check_rain_southeast_result['forecast']["forecastday"][0]['day']['daily_chance_of_rain'] == 0:
        placestogo.append('Southeast')
    if check_rain_east_result['forecast']["forecastday"][0]['day']['daily_chance_of_rain'] == 0:  
        placestogo.append('East')
    if check_rain_northeast_result['forecast']["forecastday"][0]['day']['daily_chance_of_rain'] == 0:
        placestogo.append('NorthEast')
    return placestogo

In [62]:
placestogo_list = not_raining_places()
placestogo_list 

[]

In [63]:
rainning_dir = ['North', 'East', 'West', 'South']

In [64]:
if placestogo_list == []:
    direction = st.selectbox(
    "It's raining everywhere but still, let me show you some places but I suggest to change other amount of km",
                    rainning_dir
)
else:
    direction = st.selectbox(
    'Awesome! Pick up one of the following directions or if not, get ready to get wet as it could rain bro! Choose one and see cool recomendations: ',
                    placestogo_list
)

2023-07-21 19:36:11.661 
  command:

    streamlit run C:\Users\lourdes.ruiz\miniconda3\envs\project_new_env\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [ ]:
#direction = input('Choose a direction: ')

In [ ]:
def user_direction(direction, result):
    if direction == 'north' or direction == 'N' or direction == 'North':
        return result['N']
    elif direction == 'northwest' or direction == 'NW' or direction == 'Northwest' or direction == 'NorthWest':
        return result['NO']
    elif direction == 'west' or direction == 'West' or direction == 'W':
        return result['O']
    elif direction == 'southwest' or direction == 'Southwest' or direction == 'SouthWest' or direction == 'SW':
        return result['SO']
    elif direction == 'south' or direction == direction == 'S' or direction == 'South':
        return result['S']
    elif direction == 'southeast' or direction == 'Southeast' or direction == 'SouthEast' or direction == 'SE':
        return result['SE']
    elif direction == 'east' or direction == 'East' or direction == 'E':
        return result['E']
    elif direction == 'northeast' or direction == 'Northeast' or direction == 'NorthEast' or direction == 'NE':
        return result['NE']
    else:
        return 'I am not able to find the direction'

In [ ]:
result = {'N': df_destino[df_destino['Direction'] == 'North'], 
          'NO': df_destino[df_destino['Direction'] == 'Northwest'], 
          'O': df_destino[df_destino['Direction'] == 'West'], 
          'NE': df_destino[df_destino['Direction'] == 'Northeast'],
          'S': df_destino[df_destino['Direction'] == 'South'],
          'SO': df_destino[df_destino['Direction'] == 'Southwest'],
          'SE': df_destino[df_destino['Direction'] == 'Southeast'],
          'E': df_destino[df_destino['Direction'] == 'East']}

In [ ]:
output = user_direction(direction, result)
output

In [ ]:
user_dir = output
user_dir

In [ ]:
user_direction_idx = output.index[output['Direction'] == direction][0]
user_direction_idx

In [ ]:
lat_destino = user_dir['Latitude'][user_direction_idx]
long_destino = user_dir['Longitude'][user_direction_idx]
print(lat_destino)
long_destino

In [ ]:
#Cargo el dataset de municipios para tener las coordenadas

municipios = pd.read_csv('./data/pueblos_bonitos_punto.csv')
municipios[:8]

In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers

    # Convert degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Calculate differences
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Haversine formula
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Calculate distance
    distance = R * c
    return distance

In [ ]:
municipios["Distance"] = municipios.apply(lambda row: haversine(row["Latitud"], row["Longitud"], lat_destino, long_destino), axis=1)
municipios

In [ ]:
top_3 = municipios.sort_values(by='Distance').reset_index(drop=True)[:3]
top_3

In [ ]:
one = top_3.iloc[0, top_3.columns.get_loc('Pueblo')]
two = top_3.iloc[1, top_3.columns.get_loc('Pueblo')]
three = top_3.iloc[2, top_3.columns.get_loc('Pueblo')]

In [ ]:
one

In [ ]:
two

In [ ]:
three

In [ ]:
#Llamo a la API de chatGPT aunque ahora he decidido no empezar usandola

load_dotenv()
my_secret_key = os.getenv("my_openai_key")

In [ ]:
#Llamo a la contraseña que tengo guardada en .env

openai.api_key = my_secret_key

In [ ]:
def generate_monuments_prompt(place):
    chat_prompt = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": f"List the top places to visit in {place} but\
        don't explain them and do not give me a introduction , just the answer with\
        the list. After that, let me know where I should park to sleep in a campervan.\
        Last, list me the 3 most common tracking routes and show me the amount of kilometers\
        and the difficulty level rated by easy, medium or high."}
    ]

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=chat_prompt
    )

    return response.choices[0].message.content.strip()

In [ ]:
place = one 

In [ ]:
response_text = generate_monuments_prompt(place)
print(one)
print(response_text)

In [ ]:
place = two

In [ ]:
response_text = generate_monuments_prompt(place)
print(two)
print(response_text)

In [ ]:
place = three

In [ ]:
response_text = generate_monuments_prompt(place)
print(three)
print(response_text)

In [ ]:
#print('Enjoy your trip!') 
st.write('Go for it and enjoy your trip!')

enjoy_image = Image.open('./data/enjoy.jpg')

st.image(enjoy_image)